<a href="https://colab.research.google.com/github/aishwaryasharmaccoew/CIS6930-NLP/blob/aishwarya/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LSA**

**Loading Data**

In [4]:
import pandas as pd
df1 = pd.read_csv("/content/NLP_data/depression_post_features_tfidf_256.csv")
df= df1[['post']].copy()
df.head()

,post
0,My heart aches but i cant break Recently my re...
1,Cheers I’m not sure if this is the right sub t...
2,Fuck titles. I’m depressed I feel so fucking w...
3,suffering from low energy I'm not exaggerating...
4,I can already tell This year is just going to ...


**Data Preprocessing** <br>
Steps involved are :
1. Remove all the special characters from the text.
2. Remove all the words with less than 5 letters.
3. Lowercase all the characters.
4. Remove stop words.
5. Remove profanity


In [26]:
#remove special characters
df['preprocessed_posts'] = df['post'].str.replace("[^a-zA-Z#]", " ")
#remove words have letters less than 5
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>5]))
#lowercase all characters
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: x.lower())
#remove stop words

# Remove profanity

import re
pattern = r'\b' + "fuck" + r'\w*\b'
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: re.sub(pattern, '', x))


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# tokenization
tokenized_doc = df['preprocessed_posts'].fillna('').apply(lambda x: x.split())
# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
df['preprocessed_posts'] = detokenized_doc
df['preprocessed_posts']



<ipython-input-26-06763ff2fc40>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preprocessed_posts'] = df['post'].str.replace("[^a-zA-Z#]", " ")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       recently relationship boyfriend friend attache...
1       cheers evening opened elevator really beautifu...
2       titles depressed worthless around everyone par...
3       suffering energy exaggerating taking cooking c...
4       already stupid enough optimism foolish atleast...
                              ...                        
4817    therapist walked chairs waitingroom vacant hal...
4818         family wanted school wanted working bullshit
4819    trumpet player invited finished practice every...
4820    derealization nobody really really always noth...
4821    feeling anybody living sudden feeling happened...
Name: preprocessed_posts, Length: 4822, dtype: object

**Document Term Matrix**

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
dtMatrix = vectorizer.fit_transform(df['preprocessed_posts'])
dtMatrix.get_shape()

(4822, 11900)

**Applying LSA**

In [29]:
# Universally defining the final number of expected topics
numcomp=10
topic_num=[]

for tnum in range(1,numcomp+1):
  topic_num.append("topic_"+str(tnum))

# SVD represent documents and terms in vectors 
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=numcomp, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(dtMatrix)

# Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = topic_num)
topic_encoded_df["posts"] = df['preprocessed_posts']
#display(topic_encoded_df[["posts", "topic_1", "topic_2"]])

# Features or words used as features 
dictionary = vectorizer.get_feature_names_out()

# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = topic_num, columns = (dictionary)).T


terms = vectorizer.get_feature_names_out()
for i, comp in enumerate(svd_model.components_):
    terms_in_comp = zip(dictionary, comp)
    sorted_terms = sorted(terms_in_comp, key=lambda x:x[1], reverse=True)[:10]
    topic_words = [t[0] for t in sorted_terms]
    print("Topic {}: {}".format(i+1, ", ".join(topic_words)))

Topic 1: really, people, friends, depression, things, feeling, anymore, better, depressed, school
Topic 2: people, friends, person, lonely, relationships, understand, talking, feelings, social, surrounded
Topic 3: depression, people, better, things, depressed, feeling, anxiety, thoughts, medication, change
Topic 4: depression, friends, school, family, anxiety, friend, people, parents, social, college
Topic 5: anymore, better, things, living, getting, change, trying, reason, everyday, matter
Topic 6: anymore, really, depression, people, reason, understand, shitty, tonight, feelings, diagnosed
Topic 7: depressed, feeling, anymore, friends, suicidal, friend, lonely, crying, sadness, normal
Topic 8: school, parents, college, feeling, people, getting, depressed, motivation, semester, thoughts
Topic 9: feeling, thoughts, family, friends, scared, inside, suicide, finally, suicidal, lonely
Topic 10: school, feeling, better, depression, friends, things, anymore, really, lonely, college
